# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [44]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

print(geoapify_key)

590996ee256942c08eb0b4bf3250e556


In [45]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,258.94,95,100,5.14,CA,04-02-2025
1,1,bamboo flat,11.7000,92.7167,299.20,83,1,1.03,IN,04-02-2025
2,2,blackmans bay,-43.0167,147.3167,286.22,80,7,0.45,AU,05-02-2025
3,3,talnakh,69.4865,88.3972,251.32,98,100,2.76,RU,05-02-2025
4,4,bredasdorp,-34.5322,20.0403,293.56,69,0,4.02,ZA,04-02-2025


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
print(city_data_df.shape)

# sorted_df = city_data_df.sort_values(by="Cloudiness", ascending=True)
# sorted_df.head()

narrowed_city_df = city_data_df.loc[
    # (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df.head()  #.sort_values("Cloud", ascending=True)

(578, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,bredasdorp,-34.5322,20.0403,293.56,69,0,4.02,ZA,04-02-2025
22,22,margaret river,-33.9500,115.0667,289.48,74,0,3.59,AU,05-02-2025
25,25,pisco,-13.7000,-76.2167,298.72,73,0,4.12,PE,04-02-2025
49,49,ingeniero guillermo n. juarez,-23.9000,-61.8500,316.33,15,0,2.66,AR,04-02-2025
59,59,chauvigny,46.5675,0.6493,279.36,76,0,1.21,FR,04-02-2025


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,bredasdorp,ZA,-34.5322,20.0403,69,
22,margaret river,AU,-33.9500,115.0667,74,
25,pisco,PE,-13.7000,-76.2167,73,
49,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,15,
59,chauvigny,FR,46.5675,0.6493,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
margaret river - nearest hotel: Margaret River Hotel
pisco - nearest hotel: La Portada
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
chauvigny - nearest hotel: Le Lion d'Or
adam - nearest hotel: No hotel found
la passe - nearest hotel: L'Estuaire
hawaiian paradise park - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
belmonte - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
tremp - nearest hotel: Hotel Segle XX
colonia - nearest hotel: Wasserturm Hotel Cologne
hauterive - nearest hotel: Auberge Bourbonnaise
recani - nearest hotel: Hotel Merzouga
cabo san lucas - nearest hotel: Comfort Rooms
saipan - nearest hotel: Chalan Kanoa Beach Hotel
rawson - nearest hotel: Hotel Deportivo
kutum - nearest hotel: No hotel found
tarhuna - nearest hotel: فندق ترهونة البلدي
sierra grande - nearest hotel: Sierra Grande
coahua

,City,Country,Lat,Lng,Humidity,Hotel Name
4,bredasdorp,ZA,-34.5322,20.0403,69,Victoria Hotel
22,margaret river,AU,-33.9500,115.0667,74,Margaret River Hotel
25,pisco,PE,-13.7000,-76.2167,73,La Portada
49,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,15,Hotel El Indiano
59,chauvigny,FR,46.5675,0.6493,76,Le Lion d'Or
95,adam,OM,22.3756,57.5233,38,No hotel found
105,la passe,FR,45.5549,-0.8967,70,L'Estuaire
111,hawaiian paradise park,US,19.5933,-154.9731,63,No hotel found
113,morehead city,US,34.7229,-76.7260,50,"Bask Hotel Big Rock Landing, Trademark Collect..."
130,belmonte,BR,-15.8631,-38.8828,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [50]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)